In [55]:
import pandas as pd
import numpy as np
import operator

In [17]:
training = pd.read_csv('train.data')
training = training.drop(columns='animal_name')

In [18]:
test = pd.read_csv('test.data')
test = test.drop(columns=['animal_name', 'classtype'])
test = test.values.tolist()

In [110]:
training.values

array([[1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 7],
       [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 4],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 6, 0, 0, 0, 6],
       [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 1, 0, 1, 1],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 6, 0, 0, 0, 6],
       [0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2],
       [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 7],
       [1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3]])

In [136]:
def prior_proba(column_values):
    prior_dict = {}
    length = len(column_values)
    for val in column_values:
        if val not in prior_dict.keys():
            prior_dict[val] = 1
        else:
            prior_dict[val] += 1
    
    for key, val in prior_dict.items():
        prior_dict[key] = val / length;
    return prior_dict

In [146]:
def get_column_values(matrix, i):
    return [row[i] for row in matrix]

In [157]:
list(set(get_column_values(training.values, -1)))

[1, 2, 3, 4, 6, 7]

In [160]:
def likelihood_proba(data, X_list):
    labels = get_column_values(data.values, -1)
    unique_Labels = list(set(labels))
    col_nm = data.columns
    
    
    likelihood_prob = {}
    
    for i in unique_Labels:
        class_list = []
        for j in range(0, len(X_list)):
            num = len(data.loc[(data[col_nm[j]] == X_list[j]) & (data[col_nm[-1]] == i)]) + 0.1
            den = len(data.loc[data[col_nm[-1]] == i]) + 0.1
            # Laplace Correction : 
#             if num == 0.0:
#                 num = 0.1
#                 den = den + len(np.unique(data[col_nm[j]])) + 0.1
            val = num / den
            class_list.append(val)
        likelihood_prob[i] = class_list
    return likelihood_prob

In [173]:
def posterior_proba(data, X_list):
    prior_prob = {}
    likelihood_prob = {}
    
    prior_prob = prior_proba(data.iloc[:, -1].values)
    
    likelihood_prob = likelihood_proba(data, X_list)
    
    posterior_prob = {}
    for i in prior_prob:
        n1 = prior_prob[i]
        n2 = likelihood_prob[i]
        for j in range(0, len(n2)):
            n1 = n1 * n2[j]
        posterior_prob[i] = n1
    return posterior_prob

In [174]:
def get_label(posterior_prob):
    return max(posterior_prob.items(), key=operator.itemgetter(1))

In [183]:
posterior_prob = posterior_proba(training, test[0])

{1: 0.3, 7: 0.2, 4: 0.1, 6: 0.2, 2: 0.1, 3: 0.1}


In [184]:
assert get_label(posterior_prob) == (4, 0.1)

In [185]:
expected = {1: 6.300377651627395e-13,
 7: 7.741878983954696e-08,
 4: 0.1,
 6: 1.3189542556539747e-13,
 2: 5.644739300537776e-08,
 3: 7.513148009015778e-05}

In [186]:
assert posterior_prob == expected